In [1]:
import os
import sys
import yaml
import json
import numpy as np

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [2]:
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

/Users/bcarte18/anaconda3/envs/po_pandas/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:61967 Dashboard: http://127.0.0.1:61968/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [3]:
import dask
import dask.dataframe as dd
print(dask.__version__)

2.5.0


In [4]:
%%time

nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare_dtype = dict(zip(meta_data['kaggle_nyc']['colnames'], meta_data['kaggle_nyc']['dtype']))

nyc_taxi_fare = dd.read_csv(nyc_taxi_fare_file, 
                            dtype = nyc_taxi_fare_dtype,
                             parse_dates = meta_data['kaggle_nyc']['datecols'])

CPU times: user 360 ms, sys: 11.3 ms, total: 371 ms
Wall time: 369 ms


In [5]:
%%time

nyc_taxi_fare = client.persist(nyc_taxi_fare)
progress(nyc_taxi_fare)

CPU times: user 46 ms, sys: 4.26 ms, total: 50.2 ms
Wall time: 47.8 ms


VBox()

## Check progress

[Dask Progress](http://localhost:8787/status)

In [6]:
%%time

nyc_taxi_fare.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
%%time 
# How to get the median ? 

nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].agg(["count","mean","std","min","max"]).compute()

In [18]:
del(nyc_taxi_fare)

In [41]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].describe().compute()

AttributeError: 'SeriesGroupBy' object has no attribute 'describe'

In [20]:
client.close()

In [34]:
nyc_taxi_fare.describe().fare_amount.compute()

count    10000.000000
mean        11.485460
std         10.307136
min          2.500000
25%          6.000000
50%          8.500000
75%         12.900000
max        331.290000
Name: fare_amount, dtype: float64

## DASK INTERESTING BITS ## 

In [4]:
%%time

yellow_trip_parquet = os.path.join(data_path, cfg['data']['nyc_yellow_trip_parquet_gzip'])
yellow_trip = dd.read_parquet(yellow_trip_parquet)

CPU times: user 333 ms, sys: 115 ms, total: 448 ms
Wall time: 556 ms


In [9]:
yellow_trip = client.persist(yellow_trip)
yellow_trip.dtypes

vendor_id            object
pickup_datetime      object
dropoff_datetime     object
passenger_count       int64
trip_distance       float64
                     ...   
surcharge           float64
mta_tax             float64
tip_amount          float64
tolls_amount        float64
total_amount        float64
Length: 18, dtype: object

In [10]:
import pandas as pd
pd.options.display.precision = 2
pd.options.display.max_rows = 10

In [22]:
yellow_trip.head()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('read-parquet-head-1-3-read-parquet-83fa168dd9b987b6b19676d28ac32bce', 0)", <Worker 'tcp://127.0.0.1:59951', memory: 0, processing: 1>)

In [11]:
df3 = yellow_trip.groupby('passenger_count').rate_code.count()
df3

Dask Series Structure:
npartitions=1
    int64
      ...
Name: rate_code, dtype: int64
Dask Name: series-groupby-count-agg, 3 tasks

In [ ]:
computed_df = df3.compute()
type(computed_df)